In [1]:
import pandas as pd
import numpy as np

parts_data = pd.read_csv('/Users/skylerwilson/Desktop/Lighthouse_Labs/Projects/final_project/data/Project_Data/parts_data.csv')
#parts_data.drop(columns=['Quantity Received YTD'], inplace=True)

parts_data.drop_duplicates(subset=['Part Number'], inplace=True)
parts_data = parts_data.drop(columns=['Unnamed: 0'])

In [2]:
periods = ['Part Number','Description', 'Supplier Name','Price', 'Quantity',
       'Margin %','Cost Per Unit', 'Total Cost', 'Sales - Jan', 'Sales - Feb',
       'Sales - Mar', 'Sales - Apr', 'Sales - May', 'Sales - Jun',
       'Sales - Jul', 'Sales - Aug', 'Sales - Sep', 'Sales - Oct',
       'Sales - Nov', 'Sales - Dec', 'Average Inventory', 'COGS', 'Turnover']
monthly_data = parts_data[periods]
monthly_data

KeyError: "['Cost Per Unit', 'Total Cost', 'Average Inventory', 'COGS', 'Turnover'] not in index"

In [ ]:
# Calculate demand using vectorized calculations
def calculate_demand(sales_data, alpha):
    n = len(sales_data)
    forecast = np.empty(n)
    forecast[0] = sales_data[0]
    for t in range(1, n):
        forecast[t] = alpha * sales_data[t] + (1 - alpha) * (forecast[t - 1])
    return forecast

def calculate_ordering_cost(order_quantity, fixed_ordering_costs, carrying_cost_rate, avg_lead_time_days, sales_data):
    lead_time_demand = sales_data.mean() * avg_lead_time_days
    carrying_costs_lead_time = 0.5 * carrying_cost_rate * lead_time_demand
    total_ordering_costs = fixed_ordering_costs + carrying_costs_lead_time * order_quantity
    return total_ordering_costs

# Calculate carrying cost using vectorized calculations
def calculate_carrying_cost_vectorized(average_inventory_levels, carrying_cost_rate, avg_time_in_inventory, depreciation):
    return (
        average_inventory_levels * carrying_cost_rate * avg_time_in_inventory +
        average_inventory_levels * depreciation
    )


def calculate_eoq(demand, ordering_cost, carrying_cost_month):
    if carrying_cost_month.any() == 0:
        eoq_values = np.sqrt(2 * (demand * ordering_cost))
    else:
        eoq_values = np.sqrt((2 * (demand * ordering_cost) )/ carrying_cost_month)
    return eoq_values
    
    # Calculate ordering cost for the current month
    ordering_cost = calculate_ordering_cost(
        order_quantity=order_quantity,
        fixed_ordering_costs=fixed_ordering_costs,
        carrying_cost_rate=0.125,
        avg_lead_time_days=14,
        sales_data=monthly_sales
    )

    # Calculate carrying cost using vectorized calculations
    carrying_cost = calculate_carrying_cost_vectorized(
        average_inventory_levels=np.abs(monthly_data['Average Inventory'].to_numpy()),
        carrying_cost_rate=0.125,
        avg_time_in_inventory=21,
        depreciation=0.1
    )

months = ['Sales - Jan', 'Sales - Feb', 'Sales - Mar', 'Sales - Apr', 'Sales - May', 'Sales - Jun',
          'Sales - Jul', 'Sales - Aug', 'Sales - Sep', 'Sales - Oct', 'Sales - Nov', 'Sales - Dec']


In [ ]:
monthly_data.to_csv('/Users/skylerwilson/Desktop/Lighthouse_Labs/Projects/final_project/data/Project_Data/monthly_data.csv')